<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Vector Representations
## *Data Science Unit 4 Sprint 2 Assignment 2*

In [4]:
import re
import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy
import os

from bs4 import BeautifulSoup

## 1) *Clean:* Job Listings from indeed.com that contain the title "Data Scientist" 

You have `job_listings.csv` in the data folder for this module. The text data in the description column is still messy - full of html tags. Use the [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) library to clean up this column. You will need to read through the documentation to accomplish this task. 

In [11]:
from bs4 import BeautifulSoup
import requests

##### Your Code Here #####

# get our data
df0 = pd.read_csv("data/job_listings.csv")

In [51]:
# Doing this so if I mess up the dataframe, I can just rerun this

df = df0.copy()
df.head()

,Unnamed: 0,description,title
0,0,"b""<div><div>Job Requirements:</div><ul><li><p>...",Data scientist
1,1,b'<div>Job Description<br/>\n<br/>\n<p>As a Da...,Data Scientist I
2,2,b'<div><p>As a Data Scientist you will be work...,Data Scientist - Entry Level
3,3,"b'<div class=""jobsearch-JobMetadataHeader icl-...",Data Scientist
4,4,b'<ul><li>Location: USA \xe2\x80\x93 multiple ...,Data Scientist


In [73]:
def clean_description(desc):
    soup = BeautifulSoup(desc)
    text = soup.get_text()
    if text[0] == "b":
        text=text[1:]
    return text.replace("\\n", " ")

In [74]:
df['description'] = df0['description'].apply(clean_description)

In [76]:
df.head()

,Unnamed: 0,description,title
0,0,"""Job Requirements: Conceptual understanding in...",Data scientist
1,1,"'Job Description As a Data Scientist 1, you w...",Data Scientist I
2,2,'As a Data Scientist you will be working on co...,Data Scientist - Entry Level
3,3,"'$4,969 - $6,756 a monthContractUnder the gene...",Data Scientist
4,4,'Location: USA \xe2\x80\x93 multiple locations...,Data Scientist


## 2) Use Spacy to tokenize the listings 

In [78]:
print("done 1")

# load spacy datamodel
nlp = spacy.load("en_core_web_lg")

print("done 2")

done 1
done 2


In [113]:
# add some stop words

new_stop_words = []

STOPS = nlp.Defaults.stop_words.union(new_stop_words)

#print("Added", new_stop_words, "to stop words")




['job', 'requirement', 'conceptual', 'understanding', 'Machine', 'Learning', 'model', 'like', 'nai\\xc2\\xa8ve', 'Bayes', 'K', 'Means', 'SVM', 'Apriori', 'Linear/', 'Logistic', 'Regression', 'neural', 'Random', 'Forests', 'decision', 'Trees', 'K', 'NN', 'hand', 'experience', '2', 'Intermediate', 'expert', 'level', 'coding', 'skill', 'Python', 'R.', 'ability', 'write', 'function', 'clean', 'efficient', 'datum', 'manipulation', 'mandatory', 'role', 'exposure', 'package', 'like', 'NumPy', 'SciPy', 'Pandas', 'Matplotlib', 'etc', 'Python', 'GGPlot2', 'dplyr', 'tidyR', 'r', 'ability', 'communicate', 'Model', 'finding', 'Technical', 'Non', 'technical', 'stake', 'holder', 'hand', 'experience', 'SQL', 'Hive', 'similar', 'programming', 'language', 'past', 'work', 'GitHub', 'Kaggle', 'publish', 'article', 'Master', 'degree', 'Statistics', 'Mathematics', 'Computer', 'Science', 'quant', 'specific', 'field', 'apply']


'\nfor token in q2:\n    print(token)\n'

In [114]:
# function for getting lemmas for a given text
# will be used with .apply() to get lemmas for entire dataframe

def tokenizer(text):
    
    #pacy_tokenizer = Tokenizer(nlp.vocab)
    #oc = spacy_tokenizer(text)
    
    lemmas = []
    
    doc = nlp(text)
    
    for token in doc: 
        if ((token.text.lower() not in STOPS) and (token.is_punct == False)) and (token.pos_ != 'PRON'):
            lemmas.append(token.lemma_)
        
    return lemmas

q2 = tokenizer(df['description'][0])
print(q2)


['job', 'requirement', 'conceptual', 'understanding', 'Machine', 'Learning', 'model', 'like', 'nai\\xc2\\xa8ve', 'Bayes', 'K', 'Means', 'SVM', 'Apriori', 'Linear/', 'Logistic', 'Regression', 'neural', 'Random', 'Forests', 'decision', 'Trees', 'K', 'NN', 'hand', 'experience', '2', 'Intermediate', 'expert', 'level', 'coding', 'skill', 'Python', 'R.', 'ability', 'write', 'function', 'clean', 'efficient', 'datum', 'manipulation', 'mandatory', 'role', 'exposure', 'package', 'like', 'NumPy', 'SciPy', 'Pandas', 'Matplotlib', 'etc', 'Python', 'GGPlot2', 'dplyr', 'tidyR', 'r', 'ability', 'communicate', 'Model', 'finding', 'Technical', 'Non', 'technical', 'stake', 'holder', 'hand', 'experience', 'SQL', 'Hive', 'similar', 'programming', 'language', 'past', 'work', 'GitHub', 'Kaggle', 'publish', 'article', 'Master', 'degree', 'Statistics', 'Mathematics', 'Computer', 'Science', 'quant', 'specific', 'field', 'apply']


'\nfor token in q2:\n    print(token)\n'

"Job 	 "Job
Requirements: 	 Requirements:
Conceptual 	 Conceptual
understanding 	 understanding
in 	 in
Machine 	 Machine
Learning 	 Learning
models 	 models
like 	 like
Nai\xc2\xa8ve 	 Nai\xc2\xa8ve
Bayes, 	 Bayes,
K-Means, 	 K-Means,
SVM, 	 SVM,
Apriori, 	 Apriori,
Linear/ 	 Linear/
Logistic 	 Logistic
Regression, 	 Regression,
Neural, 	 Neural,
Random 	 Random
Forests, 	 Forests,
Decision 	 Decision
Trees, 	 Trees,
K-NN 	 K-NN
along 	 along
with 	 with
hands-on 	 hands-on
experience 	 experience
in 	 in
at 	 at
least 	 least
2 	 2
of 	 of
them 	 them
Intermediate 	 Intermediate
to 	 to
expert 	 expert
level 	 level
coding 	 coding
skills 	 skills
in 	 in
Python/R. 	 Python/R.
(Ability 	 (Ability
to 	 to
write 	 write
functions, 	 functions,
clean 	 clean
and 	 and
efficient 	 efficient
data 	 data
manipulation 	 manipulation
are 	 are
mandatory 	 mandatory
for 	 for
this 	 this
role) 	 role)
Exposure 	 Exposure
to 	 to
packages 	 packages
like 	 like
NumPy, 	 NumPy,
SciPy, 	 SciPy,


In [100]:
type(q)

spacy.tokens.doc.Doc

## 3) Use Scikit-Learn's CountVectorizer to get word counts for each listing.

In [ ]:
##### Your Code Here #####

from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(stop_words='english', max_features=1000)

#Learn our Vocab
vect.fit(data)

# Get sparse dtm
dtm = vect.transform(df['description'])

dtm = pd.DataFrame(dtm.todense(), columns=vect.get_feature_names())


## 4) Visualize the most common word counts

In [ ]:
##### Your Code Here #####
raise Exception("\nThis task is not complete. \nReplace this line with your code for the task."

## 5) Use Scikit-Learn's tfidfVectorizer to get a TF-IDF feature matrix

In [ ]:
##### Your Code Here #####
raise Exception("\nThis task is not complete. \nReplace this line with your code for the task."

## 6) Create a NearestNeighbor Model. Write the description of your ideal datascience job and query your job listings. 

In [ ]:
##### Your Code Here #####
raise Exception("\nThis task is not complete. \nReplace this line with your code for the task."

## Stretch Goals

 - Try different visualizations for words and frequencies - what story do you want to tell with the data?
 - Scrape Job Listings for the job title "Data Analyst". How do these differ from Data Scientist Job Listings
 - Try and identify requirements for experience specific technologies that are asked for in the job listings. How are those distributed among the job listings?
 - Use a clustering algorithm to cluster documents by their most important terms. Do the clusters reveal any common themes?
  - **Hint:** K-means might not be the best algorithm for this. Do a little bit of research to see what might be good for this. Also, remember that algorithms that depend on Euclidean distance break down with high dimensional data.
 - Create a labeled dataset - which jobs will you apply for? Train a model to select the jobs you are most likely to apply for. :) 